In [0]:
import matplotlib.pyplot as plt
import numpy as np
import gzip
import pickle
import librosa

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Bidirectional
from keras.layers import ConvLSTM2D
from keras.utils import np_utils

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
pickle_off=gzip.open("/content/drive/My Drive/COMP562/Obxd_onenote.data","rb")
emp=pickle.load(pickle_off)

In [0]:
sampling_rate=44100
nsamples=1000;
ntrain=600;
nmel = 128
nfeatures=66*(nmel)
nparameters = 80

X=np.zeros((nsamples,nmel,66))
y=np.zeros((nsamples,nparameters))
for i in range(nsamples):
    specTemp=librosa.feature.melspectrogram(np.array(emp[i][0]), sr=sampling_rate, hop_length=1024)
    X[i,:]=(specTemp)
    y[i,:]=([emp[i][1][j][1] for j in range(0,nparameters)])

In [0]:
trainIndex=set(np.random.choice(1000,size=ntrain,replace=False))
testIndex=set(np.linspace(0,999,1000))-trainIndex
x_train=X[np.array(list(trainIndex),dtype=int),:]
y_train=y[np.array(list(trainIndex),dtype=int),:]
x_test=X[np.array(list(testIndex),dtype=int),:]
y_test=y[np.array(list(testIndex),dtype=int),:]

In [0]:
x_train=np.reshape(x_train,(ntrain,nmel,66,1))
x_test=np.reshape(x_test,(nsamples-ntrain,nmel,66,1))

In [0]:
model=Sequential()
model.add(Conv2D(64, (10, 4 ), activation='relu', input_shape=(nmel,66,1),data_format="channels_last"))
model.add(Conv2D(64, (4, 4), activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 4), activation='relu'))
model.add(Conv2D(64, (3, 4), activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(nparameters, activation='relu'))
model.add(Dense(nparameters, activation='tanh'))
model.compile(optimizer='rmsprop',
              loss='mse')
model.fit(x_train, y_train,
          epochs=100,
          batch_size=20)
score = model.evaluate(x_test, y_test, batch_size=100)
print(np.sqrt(score))


Epoch 1/100
600/600 [==============================] - 2s 3ms/step - loss: 0.2967
Epoch 2/100
600/600 [==============================] - 1s 1ms/step - loss: 0.0994
Epoch 3/100
600/600 [==============================] - 1s 1ms/step - loss: 0.0877
Epoch 4/100
600/600 [==============================] - 1s 1ms/step - loss: 0.0873
Epoch 5/100
600/600 [==============================] - 1s 1ms/step - loss: 0.0879
Epoch 6/100
600/600 [==============================] - 1s 1ms/step - loss: 0.0872
Epoch 7/100
600/600 [==============================] - 1s 1ms/step - loss: 0.0863
Epoch 8/100
600/600 [==============================] - 1s 1ms/step - loss: 0.0864
Epoch 9/100
600/600 [==============================] - 1s 1ms/step - loss: 0.0867
Epoch 10/100
600/600 [==============================] - 1s 1ms/step - loss: 0.0854
Epoch 11/100
600/600 [==============================] - 1s 1ms/step - loss: 0.0856
Epoch 12/100
600/600 [==============================] - 1s 1ms/step - loss: 0.0852
Epoch 13/100


In [0]:
pickle_off=gzip.open("/content/drive/My Drive/COMP562/NSynth.data","rb")
nsynth = pickle.load(pickle_off, encoding='latin1')

In [0]:
nsynth_processed=np.zeros((len(nsynth),nmel,66))
for i in range(len(nsynth)):
    specTemp=librosa.feature.melspectrogram(np.append(nsynth[i],np.zeros(1000)), sr=sampling_rate, hop_length=1024)
    nsynth_processed[i,:]=(specTemp)
nsynth_processed=np.reshape(nsynth_processed,(len(nsynth),nmel,66,1))


In [0]:
results = model.predict(nsynth_processed)

In [0]:
datafile = gzip.open("/content/drive/My Drive/COMP562/Obxd_CNN_NSynth.data","wb")
pickle.dump(results, datafile)
datafile.close()

# Additional Models

In [0]:
model=Sequential()
model.add(Conv2D(64, (4, 4), activation='relu', input_shape=(nmel,66,1),data_format="channels_last"))
model.add(Conv2D(64, (4, 4), activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(nparameters, activation='relu'))
model.add(Dense(nparameters, activation='tanh'))
model.compile(optimizer='rmsprop',
              loss='mse')
model.fit(x_train, y_train,
          epochs=200,
          batch_size=20)
score = model.evaluate(x_test, y_test, batch_size=100)
print(np.sqrt(score))

In [0]:
model=Sequential()
model.add(Conv2D(128, (24, 4), activation='relu', input_shape=(nmel,66,1),data_format="channels_last"))
model.add(Conv2D(128, (4, 4), activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 4), activation='relu'))
model.add(Conv2D(64, (3, 4), activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(nparameters, activation='relu'))
model.add(Dense(nparameters, activation='tanh'))
model.compile(optimizer='rmsprop',
              loss='mse')
model.fit(x_train, y_train,
          epochs=10,
          batch_size=20)
score = model.evaluate(x_test, y_test, batch_size=100)
print(np.sqrt(score))